In [1]:
import tensorflow as tf
# TensorFlow에서 사용할 수 있는 GPU 확인
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"사용 가능한 GPU 장치: {gpus}")
else:
    print("사용 가능한 GPU가 없습니다.")

# 간단한 행렬 연산을 통해 GPU 사용 여부 테스트
with tf.device('/GPU:0'):  # GPU:0은 첫 번째 GPU를 의미합니다.
    a = tf.random.normal([10000, 10000])
    b = tf.random.normal([10000, 10000])
    result = tf.matmul(a, b)
    print(result)
print("연산 완료.")

2024-09-09 12:01:56.349093: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-09 12:01:58.049388: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdirectml.d6f03b303ac3c4f2eeb8ca631688c9757b361310.so
2024-09-09 12:01:58.049501: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdxcore.so
2024-09-09 12:01:58.053532: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libd3d12.so
2024-09-09 12:01:58.273191: I tensorflow/c/logging.cc:34] DirectML device enumeration: found 1 compatible adapters.


사용 가능한 GPU 장치: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-09-09 12:01:59.017681: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-09 12:01:59.019110: I tensorflow/c/logging.cc:34] DirectML: creating device on adapter 0 (AMD Radeon(TM) RX Vega 11 Graphics)
2024-09-09 12:01:59.130387: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-09 12:01:59.130459: W tensorflow/core/common_runtime/pluggable_device/pluggable_device_bfc_allocator.cc:28] Overriding allow_growth setting because force_memory_growth was requested by the device.
2024-09-09 12:01:59.130496: I tensorflow/core/common_runtime/pluggable_device/pluggab

tf.Tensor(
[[ -71.59939    -55.189156    74.72947   ...  223.25272   -189.48872
   -94.077705 ]
 [ -54.31707    -73.3139      47.46685   ...  -16.11756    122.02861
   -11.460805 ]
 [  66.01808    -72.25567    -89.45663   ...  161.80592     -5.8517227
    82.13046  ]
 ...
 [  16.878582  -107.460594  -179.46054   ...  166.95715    115.20126
   -88.887054 ]
 [-106.95812   -144.9183     -62.150978  ...  -51.68599     -9.329445
    -4.7425528]
 [  92.86312    -71.35511   -131.79999   ...   84.50442    -95.055855
  -106.29631  ]], shape=(10000, 10000), dtype=float32)
연산 완료.


In [2]:
import numpy as np

In [3]:
print(np.__version__)

1.26.4


In [ ]:
!pwd

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# 이미지 불러오기
img_path = '/mnt/c/Users/5-29/Downloads/1612312679152－2.jpg'
img = Image.open(img_path)
img = img.resize((480, 640))  # 이미지를 640x480 크기로 변경

# 이미지를 NumPy 배열로 변환
img_array = np.array(img).astype('float32') / 255.0
img_array = img_array.reshape((1, img_array.shape[0], img_array.shape[1], img_array.shape[2]))  # 배치 차원 추가

# Box 블러 필터 정의 (3x3 필터)
box_blur_kernel = np.array([[1/9, 1/9, 1/9],
                            [1/9, 1/9, 1/9],
                            [1/9, 1/9, 1/9]], dtype='float32')

# Box 블러 필터의 모양을 (3, 3, 3, 3)로 맞춤
# 3x3 크기 필터, 3개의 입력 채널(RGB), 3개의 출력 채널을 각각 설정
box_blur_kernel_rgb = np.stack([box_blur_kernel]*3, axis=-1)  # 각 채널에 대해 동일한 필터 적용
box_blur_kernel_rgb = np.stack([box_blur_kernel_rgb]*3, axis=-1)  # 출력 채널에 대해서도 동일하게 설정

# 입력을 위한 텐서플로우 레이어 생성
input_layer = tf.keras.layers.Input(shape=(640, 480, 3))

# Box 블러 필터 적용
blur_layer = tf.keras.layers.Conv2D(3, (3, 3), padding='same', kernel_initializer=tf.constant_initializer(box_blur_kernel_rgb))(input_layer)

# 모델 구성
model = tf.keras.models.Model(inputs=input_layer, outputs=blur_layer)

# 필터 적용 결과 확인
blur_output = model.predict(img_array)

# 출력 이미지의 값을 [0, 1] 사이로 정규화
blur_output_normalized = (blur_output[0] - blur_output[0].min()) / (blur_output[0].max() - blur_output[0].min())

# 결과 시각화
plt.figure(figsize=(10, 5))

# 원본 이미지 시각화
plt.subplot(1, 2, 1)
plt.imshow(img)
plt.title('Original Image')
plt.axis('off')

# 블러 처리된 이미지 시각화
plt.subplot(1, 2, 2)
plt.imshow(blur_output_normalized)  # 정규화된 이미지를 표시
plt.title('Blurred Image')
plt.axis('off')

plt.show()